In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import torch.optim as optim

In [2]:
USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda' if USE_CUDA else 'cpu')
print("다음기기 사용:", device)

다음기기 사용: cpu


In [3]:
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(7777)

In [4]:
training_epochs = 15
batch_size = 100

In [5]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root="MNIST_data/",
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [6]:
data_loader = DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True,drop_last=True)

In [7]:
data_loader

In [8]:
linear = nn.Linear(784, 10, bias=True).to(device)

In [9]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(linear.parameters(), lr=0.1)

In [10]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(data_loader)

    for X, Y in data_loader:
        X=X.view(-1, 28*28).to(device)
        Y=Y.to(device)

        optimizer.zero_grad()
        y_hat = linear(X)
        cost = criterion(y_hat, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost/total_batch

    print('Epoch:', epoch, 'Cost:', avg_cost)
print('Learning finished')


Epoch: 0 Cost: tensor(0.5352, grad_fn=<AddBackward0>)
Epoch: 1 Cost: tensor(0.3596, grad_fn=<AddBackward0>)
Epoch: 2 Cost: tensor(0.3313, grad_fn=<AddBackward0>)
Epoch: 3 Cost: tensor(0.3164, grad_fn=<AddBackward0>)
Epoch: 4 Cost: tensor(0.3071, grad_fn=<AddBackward0>)
Epoch: 5 Cost: tensor(0.3005, grad_fn=<AddBackward0>)
Epoch: 6 Cost: tensor(0.2949, grad_fn=<AddBackward0>)
Epoch: 7 Cost: tensor(0.2910, grad_fn=<AddBackward0>)
Epoch: 8 Cost: tensor(0.2871, grad_fn=<AddBackward0>)
Epoch: 9 Cost: tensor(0.2845, grad_fn=<AddBackward0>)
Epoch: 10 Cost: tensor(0.2819, grad_fn=<AddBackward0>)
Epoch: 11 Cost: tensor(0.2795, grad_fn=<AddBackward0>)
Epoch: 12 Cost: tensor(0.2776, grad_fn=<AddBackward0>)
Epoch: 13 Cost: tensor(0.2759, grad_fn=<AddBackward0>)
Epoch: 14 Cost: tensor(0.2744, grad_fn=<AddBackward0>)
Learning finished


In [ ]:
with torch.no_grad():
    x_test=mnist_test.test_data.view(-1, 28*28).float().to(device)
    y_test = mnist_test.test_labels.to(device)

    prediction = linear(x_test)
    correct_prediction = torch.argmax(prediction, 1)==y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())
    r = random.randint(0, len(mnist_test) -1)
    x_single_data = mnist_test.test_data[r:r+1].view(-1, 28*28).float().to(device)
    y_single_data = mnist_test.test_labels[r:r+1].to(device)
    print('Label:',y_single_data)
    single_prediction = linear(x_single_data)
    print(single_prediction)
    print('Prediction:', torch.argmax(single_prediction,1).item())

    plt.imshow(mnist_test.test_data[r:r+1].view(28,28), cmap='Grays')
    plt.show()
